# Imports

In [ ]:
import os
import numpy as np
import torch
from datasets import load_dataset, load_metric
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer
)
from torchvision.transforms import (
    Compose, Normalize, RandomHorizontalFlip,
    RandomResizedCrop, Resize, ToTensor
)

# --- 1. CONFIGURAÇÕES INICIAIS ---

In [ ]:
# Nome do modelo base (Vision Transformer - SOTA para classificação)
MODEL_CHECKPOINT = "google/vit-base-patch16-224-in21k"
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 5  # 5 épocas costuma ser bom para começar com Transfer Learning
DATASET_DIR = "dataset_soja" # Nome da pasta onde estão as subpastas (Caterpillar, etc)

Configurar dispositivo (GPU se disponível, senão CPU)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🌲 Iniciando projeto SojaTech usando: {device}")

# --- 2. CARREGAMENTO E DIVISÃO DOS DADOS ---

In [ ]:
print("📂 Carregando imagens...")
# A biblioteca 'datasets' lê a estrutura de pastas automaticamente
dataset = load_dataset("imagefolder", data_dir=DATASET_DIR)

: 

Criar mapeamento de Labels (Ex: 0 -> Caterpillar, 1 -> Saudavel...)

In [ ]:
labels = dataset['train'].features['label'].names
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}
num_labels = len(labels)
print(f"Classes encontradas: {labels}")

Divisão Estratificada (Crucial devido ao desbalanceamento das classes)
80% Treino / 20% Validação

In [ ]:
splits = dataset['train'].train_test_split(test_size=0.2, stratify_by_column="label", seed=42)
train_ds = splits['train']
val_ds = splits['test']

print(f"Imagens de Treino: {len(train_ds)}")
print(f"Imagens de Validação: {len(val_ds)}")

# --- 3. PRÉ-PROCESSAMENTO (TRANSFORMS) ---
Carregar o processador original do modelo

In [ ]:
processor = AutoImageProcessor.from_pretrained(MODEL_CHECKPOINT)

Definir normalização (média e desvio padrão do ImageNet)

In [ ]:
normalize = Normalize(mean=processor.image_mean, std=processor.image_std)

Transformações: O modelo espera 224x224, mas as tuas imagens são 500x500.
Vamos redimensionar e aplicar Data Augmentation suave.

In [ ]:
_train_transforms = Compose([
    RandomResizedCrop(processor.size['height']), # Recorte aleatório e resize para 224
    RandomHorizontalFlip(),                      # Espelhamento (ajuda na generalização)
    ToTensor(),
    normalize,
])

_val_transforms = Compose([
    Resize(processor.size['height']),            # Apenas redimensiona para 224
    ToTensor(),
    normalize,
])

def transform_train(examples):
    examples["pixel_values"] = [_train_transforms(img.convert("RGB")) for img in examples["image"]]
    return examples

def transform_val(examples):
    examples["pixel_values"] = [_val_transforms(img.convert("RGB")) for img in examples["image"]]
    return examples

Aplicar as transformações ao dataset

In [ ]:
train_ds.set_transform(transform_train)
val_ds.set_transform(transform_val)

# --- 4. DEFINIÇÃO DO MODELO ---

In [ ]:
print("🤖 Baixando e configurando o modelo...")
model = AutoModelForImageClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True # Necessário pois estamos mudando o número de classes final
)

# --- 5. CONFIGURAÇÃO DO TREINO ---

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    """Calcula a acurácia das previsões"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

args = TrainingArguments(
    output_dir="resultado_soja",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2, # Salva apenas os 2 melhores checkpoints para economizar espaço
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor,
    compute_metrics=compute_metrics,
)

# --- 6. EXECUTAR TREINO E SALVAR ---

In [ ]:
print("🚀 Iniciando treinamento...")
trainer.train()

print("💾 Salvando modelo final...")
trainer.save_model("modelo_soja_final")
processor.save_pretrained("modelo_soja_final")

# Avaliação Final
metrics = trainer.evaluate()
print("-" * 30)
print("RESULTADOS FINAIS:")
print(f"Acurácia: {metrics['eval_accuracy']:.2%}")
print("-" * 30)